In [260]:
import pandas as pd
import numpy as np
import os

In [ ]:
os.chdir('/content/drive/MyDrive/Machine Learning Algo Implementations/Decision Trees')

In [261]:
# function to find the gini index corresponding to a given split 
# given the target column

def gini_index(y_col):
  present_classes,counts=np.unique(y_col,return_counts=True)
  N=y_col.shape[0]

  prob_squared=0.0

  for count in counts:
    prob_squared+=(count/N)**2
  
  gini=1-prob_squared
  return gini

In [262]:
df.head()

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.3,1.7,0.5
1,5.1,3.7,1.5,0.4
2,7.2,3.2,6.0,1.8
3,6.0,2.7,5.1,1.6
4,5.4,3.0,4.5,1.5


In [263]:
# this function splits the train_data into two child nodes depending on the 
# 'feature' and it's value. If feature_value < value then this row goes into 
# left child else it goes into right child 

def split_data(x_train,y_train,feature,value):

  x_left_split=pd.DataFrame(columns=x_train.columns)
  x_right_split=pd.DataFrame(columns=x_train.columns)
  y_left_split=[]
  y_right_split=[]

  for index,row in x_train.iterrows():
    if(row[feature]<value):
      x_left_split.loc[index]=row
      y_left_split.append(y_train[index])

    else:
      x_right_split.loc[index]=row
      y_right_split.append(y_train[index])
  
  y_left_split=np.array(y_left_split)
  y_right_split=np.array(y_right_split)
  
  return x_left_split,y_left_split,x_right_split,y_right_split

In [264]:
# function gives the best gini index and corresponding split point 
# for a particular feature 

def split_feature_helper(x_train,y_train,feature):

  col=x_train[feature].values
  sorted_col=sorted(col)
  curr_best_gini=999
  curr_best_split_point=-1
  N=x_train.shape[0]

  for index in range(len(sorted_col)-1):
    split_point=(sorted_col[index]+sorted_col[index+1])/2

    # splitting the x_train,y_train into two halfs for left x_train[feature]<split_point
    # for right half x_train[feature]>=split_point 
    
    x_left_split,y_left_split,x_right_split,y_right_split=split_data(x_train,y_train,feature,split_point)
    left_gini=gini_index(y_left_split)
    right_gini=gini_index(y_right_split)
    curr_gini=(x_left_split.shape[0]/N)*(left_gini)+(x_right_split.shape[0]/N)*(right_gini)

    if(curr_gini<curr_best_gini):
      curr_best_gini=curr_gini
      curr_best_split_point=split_point

  return curr_best_gini,curr_best_split_point  

In [265]:
# function is used to get the split feature,split point and gini index 

def split_feature(x_train,y_train):

  feature_names=x_train.columns
  best_gini=999
  best_split_point=-1
  best_feature=None

  for feature in feature_names:
    curr_best_gini,curr_best_split_point=split_feature_helper(x_train,y_train,feature)
    ## print('feature:',feature,'best_split_point:',curr_best_split_point,
    ##      'best_gini:',curr_best_gini)

    if(curr_best_gini<best_gini):
      best_gini=curr_best_gini
      best_feature=feature
      best_split_point=curr_best_split_point

  return best_feature,best_split_point,best_gini

In [266]:
class node:
  def __init__(self,depth,class_distribution):
    self.left=None
    self.right=None
    self.prediction=None
    self.is_leaf=False  
    self.depth=depth
    self.split_point=-1
    self.gini=0
    self.split_feature=None
    self.class_distribution=class_distribution

  def print_node(self):
    
    if(self.is_leaf):
      print('Level:',self.depth)
      print('We are at leaf node')
      print('Gini index:',self.gini)
      print('Class Wise Distribution:',self.class_distribution)
      print('Class Prediction:',self.prediction)

    else:
      print('Level:',self.depth)
      print('Feature To Split On:',self.split_feature,' <',round(self.split_point,3)) 
      print('Gini index:',self.gini)
      print('Class Wise Distribution:',self.class_distribution)

    print('------------------------------------------------------')

In [267]:
def build_tree(x_train,y_train,depth,max_depth):

  classes,counts=np.unique(y_train,return_counts=True)
  class_distribution=dict(zip(classes,counts))
  root=node(depth,class_distribution)

  most_occuring_class_index=np.argmax(counts)
  class_name=classes[most_occuring_class_index]
  count=counts[most_occuring_class_index]
  
  # if a node is pure node
  if(count==x_train.shape[0]):
    root.is_leaf=True
    root.prediction=classes[np.argmax(counts)]

  # if the current depth of the tree is equal to max depth we stop 
  elif(depth==max_depth):
    root.is_leaf=True
    root.prediction=classes[np.argmax(counts)]

  elif(depth>max_depth):
    return None

  else:
    best_feature,best_split_point,best_gini=split_feature(x_train,y_train)
    x_left_split,y_left_split,x_right_split,y_right_split=split_data(x_train,y_train,best_feature,best_split_point)

    root.split_point=best_split_point
    root.gini=best_gini
    root.split_feature=best_feature

    x_left_split.reset_index(drop=True,inplace=True)
    x_right_split.reset_index(drop=True,inplace=True)

    
    # recursively work on the left and right splits
    root.left=build_tree(x_left_split,y_left_split,depth+1,max_depth)
    root.right=build_tree(x_right_split,y_right_split,depth+1,max_depth)

  return root

In [268]:
def print_tree(root):
  if(root==None):
    return

  root.print_node()

  print_tree(root.left)
  print_tree(root.right)

In [291]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [270]:
raw_df=pd.read_csv('iris.csv')
raw_df=raw_df.sample(frac=1).reset_index(drop=True)

In [271]:
df=raw_df.copy()

In [272]:
Y=df['variety'].values
df.drop('variety',axis=1,inplace=True)

In [273]:
train_df,test_df,y_train,y_test=train_test_split(df,Y)

In [274]:
train_df.shape,y_train.shape,test_df.shape,y_test.shape

((112, 4), (112,), (38, 4), (38,))

In [275]:
train_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

In [276]:
x_train=train_df.values
x_test=test_df.values

In [277]:
root=build_tree(train_df,y_train,1,10)

In [278]:
print_tree(root)

Level: 1
Feature To Split On: petal.length  < 2.45
Gini index: 0.31919642857142866
Class Wise Distribution: {'Setosa': 40, 'Versicolor': 33, 'Virginica': 39}
------------------------------------------------------
Level: 2
We are at leaf node
Gini index: 0
Class Wise Distribution: {'Setosa': 40}
Class Prediction: Setosa
------------------------------------------------------
Level: 2
Feature To Split On: petal.width  < 1.75
Gini index: 0.07933436532507727
Class Wise Distribution: {'Versicolor': 33, 'Virginica': 39}
------------------------------------------------------
Level: 3
Feature To Split On: petal.length  < 5.05
Gini index: 0.0392156862745098
Class Wise Distribution: {'Versicolor': 32, 'Virginica': 2}
------------------------------------------------------
Level: 4
We are at leaf node
Gini index: 0
Class Wise Distribution: {'Versicolor': 31}
Class Prediction: Versicolor
------------------------------------------------------
Level: 4
Feature To Split On: sepal.length  < 6.15
Gini in

In [279]:
def predict(root,test_data):
  
  if(root.is_leaf):
    return root.prediction

  if(test_data[root.split_feature]>=root.split_point):
    return predict(root.right,test_data)
  
  else:
    return predict(root.left,test_data)

In [280]:
predictions=[]
for _,row in test_df.iterrows():  
  res=predict(root,row)
  predictions.append(res)

In [287]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

      Setosa       1.00      1.00      1.00        10
  Versicolor       0.85      1.00      0.92        17
   Virginica       1.00      0.73      0.84        11

    accuracy                           0.92        38
   macro avg       0.95      0.91      0.92        38
weighted avg       0.93      0.92      0.92        38



In [292]:
print(confusion_matrix(y_test,predictions))

[[10  0  0]
 [ 0 17  0]
 [ 0  3  8]]


## Using Sklearn Deicion Tree

In [282]:
from sklearn.tree import DecisionTreeClassifier

In [283]:
clf=DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [284]:
clf.score(x_test,y_test)

0.9210526315789473